In [1]:
num_sample = 300
num_burn = 100
sample_size = 800

In [2]:
from dgp import sample_network_chain, get_graph, sample_Y1, sample_Y2, agcEffect
import numpy as np

# 1. Simulate or load data
min_degree = 4
max_degree = 8
adj = get_graph(sample_size, min_degree, max_degree, seed=1)
print(adj.sum(axis=1).min(), adj.sum(axis=1).max())
#adj = np.zeros((sample_size, sample_size))

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-2.00,0.50,-0.05,0.20,0.05,0.25,-0.08,0.30])     # shape (8,)
beta = np.array([-0.30,-0.60,-0.60,-0.20,-0.05,-0.10,-0.01,0.40,0.01,0.20])     # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain(adj, tau, rho, nu, gamma, beta, R=num_sample,
    burnin_R=num_burn, seed=0, sample_Y_func=sample_Y2, Atype=('gen', 0.7))

Y_chain = Y_chain[::3]
A_chain = A_chain[::3]
L_chain = L_chain[::3]

dir, dir2 = [], []
for i in range(Y_chain.shape[0]):
    Y = Y_chain[i]
    A = A_chain[i]
    L = L_chain[i]
    dir.append(np.mean(Y[A==1]) - np.mean(Y[A==0]))
    dir2.append(np.mean(A))

np.mean(dir), np.mean(dir2)

4 8


100%|██████████| 40000/40000 [02:00<00:00, 331.10it/s]


(-0.029023643877685083, 0.25877275)

In [3]:
import networkx as nx
def compute_avg_effects_std_from_raw(psi_vec, adj_matrix, h=2):
    N = len(psi_vec)
    avg_effects = np.mean(psi_vec)
    
    # Centered residuals
    g = psi_vec - avg_effects
    
    # Build graph and compute pairwise distances
    G = nx.from_numpy_array(adj_matrix)
    dist = dict(nx.all_pairs_shortest_path_length(G, cutoff=h))
    
    # Network HAC estimator
    hac_var = 0.0
    for i in range(N):
        for j, dij in dist[i].items():
            weight = max(1 - dij / h, 0)  # Bartlett kernel
            hac_var += weight * g[i] * g[j]
    
    hac_var /= N
    se_hac = np.sqrt(hac_var)/np.sqrt(N)

    return avg_effects, se_hac

In [ ]:
from run_pll import *
from utils import run_pll

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr_raw, args, processes=10)
res_list_array_dr = np.array(res_list_dr)

In [ ]:
from tqdm import tqdm

var = []
var_sim = []
est = []
idx = 0
for i in tqdm(range(Y_chain.shape[0])):
    avg_effects, se_hac = compute_avg_effects_std_from_raw(res_list_dr[i,:,0],adj,h=3)
    var.append(se_hac**2*sample_size)
    var_sim.append(np.var(res_list_dr[i,:,0]))
    est.append(avg_effects)

var_true = np.var(est)
print(np.mean(var), np.mean(var_sim), var_true*sample_size)

100%|██████████| 10000/10000 [06:58<00:00, 23.91it/s]

0.2309678569227906 0.23268793578125 0.2395156434875


In [5]:
# compute coverage rate for mean(Y)

mean_est = np.mean(Y_chain, axis=1)
#std_est = np.sqrt(np.array(var))
std_est = np.std(Y_chain, axis=1)
se_est = std_est / np.sqrt(Y_chain.shape[1])
lower_bound = mean_est - 1.96*se_est
upper_bound = mean_est + 1.96*se_est
truth = np.mean(Y_chain)
coverage_rate = np.mean((lower_bound < truth) & (upper_bound > truth))
print("Coverage rate for mean(Y):", coverage_rate)

Coverage rate for mean(Y): 0.8722


In [6]:
np.sqrt(np.array(var)).mean(), np.std(Y_chain, axis=1).mean(), np.mean(var), np.var(Y_chain, axis=1).mean()

(0.4794930129108512,
 0.36706748125934646,
 0.2309678569227906,
 0.13497900359374998)

In [7]:
k = 51
print(np.where(adj[k])[0])
for j in np.where(adj[k])[0]:
    print(np.corrcoef(L_chain[:,k,0], L_chain[:,j,0])[0,1])

ic = []
for j in np.where(adj[0]==0)[0]:
    ic.append(np.corrcoef(L_chain[:,k,0], L_chain[:,j,0])[0,1])
np.mean(ic)

[36 44 48 50 53 54 57 62]
-0.0024047659344970016
-0.006200550184607464
0.0002895150593442676
-0.0058514365867054765
0.003913545804805268
0.009396890404550115
-0.005531425003369036
-0.013987499018662464


0.0017564936591066714

In [8]:
L_chain.shape, Y_chain.shape

((10000, 800, 3), (10000, 800))